# Retrieve Dataset

In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np;

raw_ds = tfds.load('deep_weeds', split="train", as_supervised=True)


2021-10-10 11:39:59.813791: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Pre-process Dataset

In [4]:
def negative_class_filter(x, y):
  return tf.not_equal(y, 8)

def rgb_to_grayscale_map(x, y):
  return (tf.reduce_mean(x, 2), y)

full_ds = raw_ds.filter(negative_class_filter) # remove instances from negative class
full_ds_size = full_ds.reduce(0, lambda x,_: x + 1).numpy()
full_ds = full_ds.shuffle(full_ds_size) # shuffle dataset
full_ds = full_ds.map(rgb_to_grayscale_map) # convert to grayscale

2021-10-10 11:40:03.985302: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


# Train Test Split

In [39]:
train_size = int(0.7 * full_ds_size)
test_size = int(0.3 * full_ds_size)

# train_size = 1000
# test_size = 1000
used_size = train_size + test_size

used_ds = full_ds.take(used_size)

train_ds = used_ds.take(train_size)
test_ds = used_ds.skip(train_size)

train_ds_numpy = tfds.as_numpy(train_ds)
test_ds_numpy = tfds.as_numpy(test_ds)

print(f'Total number of all instances = {full_ds_size}')
print(f'Total number of used instances = {used_size}')
print(f'Total number of training instances = {train_size}')
print(f'Total number of testing instances = {test_size}')

# TODO: Reduce dimension
# TODO: Abstract function

Total number of all instances = 8403
Total number of used instances = 8402
Total number of training instances = 5882
Total number of testing instances = 2520


# k-Fold Cross Validation

In [31]:
def get_accuracy(predicted_Y, actual_Y):
  total_count = len(predicted_Y)
  correct_count = 0
  for i in range(total_count):
    if predicted_Y[i] == actual_Y[i]:
      correct_count += 1
  accuracy = correct_count / total_count
  return accuracy

In [32]:
train_X = []
train_Y = []
for ex in test_ds_numpy:
  x = ex[0].flatten()
  y = ex[1]
  train_X.append(x)
  train_Y.append(y)
print(f'Sample train_X: {train_X[0]}')
print(f'Sample train_Y: {train_Y[0]}')

Sample train_X: [251 250 111 ...  10  17  18]
Sample train_Y: 3


In [34]:
from sklearn.model_selection import KFold
from sklearn import tree

k = 5
kf = KFold(n_splits=k)

accuracy_scores = []

for train_index, test_index in kf.split(train_X):
  cf_train_X = [train_X[index] for index in train_index]
  cf_train_Y = [train_Y[index] for index in train_index]
  cf_test_X = [train_X[index] for index in test_index]
  cf_test_Y = [train_Y[index] for index in test_index]
  clf = tree.DecisionTreeClassifier()
  clf = clf.fit(cf_train_X, cf_train_Y)
  predicted_Y = clf.predict(cf_test_X)
  accuracy = get_accuracy(predicted_Y, cf_test_Y)
  accuracy_scores.append(accuracy)

average_accuracy = sum(accuracy_scores) / k
print(f'Accuracy for each fold: {accuracy_scores}')
print(f'Average accuracy: {average_accuracy}')

Accuracy for each fold: [0.16, 0.215, 0.19, 0.135, 0.16]
Average accuracy: 0.172


# Train Model (Decision Tree)

In [38]:

# ~170s for 1000 training instances max_depth=None RGB

# ~42.2s for 1000 training instances max_depth=None Grayscale

# ~89.5s for 1000 training instances max_depth=5 RGB

# Evaluate Model

In [37]:
# 0.29 ~85.3s for 1000 training instances max_depth=None RGB

# 0.266 ~0.2s for 1000 training instances max_depth=None Grayscale

# 0.216 ~68.4s for 1000 training instances max_depth=5 RGB

In [35]:
# test_X = []
# test_Y = []
# for ex in test_ds_numpy:
#   x = ex[0].flatten() # 256 x 256 (Grayscale)
#   y = ex[1]
#   test_X.append(x)
#   test_Y.append(y)
# print(f'Sample test_X: {test_X[0]}')
# print(f'Sample test_Y: {test_Y[0]}')

Sample test_X: [104  71  43 ... 197 248  45]
Sample test_Y: 0
